In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

# --- Configuration for Data Generation ---
NUM_TOP_LEVEL_TASKS = 50
MAX_SUB_LEVELS = 3 # Max depth of subtasks (e.g., Project -> Feature -> Task)
TASKS_PER_LEVEL_RANGE = (1, 4) # Min and max subtasks per parent
CHANCE_TO_HAVE_SUBTASKS = 0.7 # Probability a parent task will have subtasks
CHANCE_TO_HAVE_PREDECESSOR = 0.2 # Probability a task has a predecessor


# --- Data Value Options ---
# Refined Project Names to sound like actual initiatives
PROJECT_NAMES = [
    "Q3 Product Feature Launch - AI Integration",
    "Holiday Marketing Campaign - Social Media Focus",
    "Website Redesign - User Experience Overhaul",
    "Internal Tool Development - Automation Suite",
    "Customer Onboarding Flow Optimization"
]

# Refined Task Types to reflect phases/roles in a workflow
TASK_TYPES_LIST = [
    'Strategy & Planning', 'Design & UX', 'Content Creation',
    'Legal & Compliance Review', 'Development (Frontend)', 'Development (Backend)',
    'QA & Testing', 'Deployment & Release', 'Marketing & Promotion',
    'Analytics & Reporting', 'PMO & Coordination', 'Customer Support Prep'
]


# Assignee details (for dim_assignees) - Keep as is, they are generic enough
ASSIGNEE_TEAMS = ['Team Alpha', 'Team Beta', 'Team Gamma', 'Team Delta']
ASSIGNEE_NAMES_PER_TEAM = {
    'Team Alpha': ['Alice', 'Bob', 'Charlie'],
    'Team Beta': ['David', 'Eve', 'Frank'],
    'Team Gamma': ['Grace', 'Heidi', 'Ivan'],
    'Team Delta': ['Judy', 'Karl', 'Liam']
}

ASSIGNEE_COST_PER_HOUR_RANGE = (20.0, 50.0)

# Task type values (for dim_task_types) - Keep as is, values are fine
ESTIMATED_TASK_VALUE_RANGE = (50.0, 500.0)

# Status options - Keep as is, these are standard
STATUS_OPTIONS = ['To Do', 'In Progress', 'Blocked', 'Review', 'Completed']
PRIORITY_OPTIONS = ['Critical', 'High', 'Medium', 'Low']

# Refined Blocker Reasons to be more specific to workflows
BLOCKER_REASONS = [
    'Awaiting Client Feedback', 'Resource Unavailability', 'Unexpected Scope Creep',
    'Technical Bug Found', 'External API Dependency', 'Legal Review Pending',
    'Design Approval Needed', 'Cross-Team Handoff Delay', 'Data Access Issues'
]
QUALITY_REVIEW_STATUS_OPTIONS = ['Passed', 'Minor Rework', 'Major Rework', 'Failed QA']


# --- Messiness Configuration (for Version A) - Keep as is, these rates are good
MISSING_VALUE_RATES = {
    'due_date': 0.15,
    'time_tracked_hours': 0.20,
    'blocker_reason': 0.70,
    'date_completed': 0.05,
    'percentage_completion': 0.10
}
INCONSISTENT_SPELLING_RATES = {
    'status': 0.25,
    'blocker_reason': 0.30,
    'priority': 0.15
}
ILLOGICAL_DATA_RATES = {
    'completed_but_not_100_percent': 0.05,
    'in_progress_but_100_percent': 0.03,
    'completed_before_created': 0.02,
    'overdue_but_not_blocked': 0.05
}


# --- Update the task_name generation logic in generate_task_data function ---
# This part is inside the generate_task_data function, not in the config block
# Original: 'task_name': f"Task {task_id} - {random.choice(['Feature', 'Bug Fix', 'Report', 'Analysis', 'Meeting Prep'])}",
# New:
TASK_NAME_SUFFIXES = {
    'Strategy & Planning': ['Define Objectives', 'Market Research', 'Budget Allocation'],
    'Design & UX': ['Wireframe Creation', 'Mockup Design', 'User Testing'],
    'Content Creation': ['Ad Copy Draft', 'Social Media Post', 'Blog Article'],
    'Legal & Compliance Review': ['Review Ad Copy', 'Compliance Check', 'Contract Approval'],
    'Development (Frontend)': ['Build UI Component', 'Integrate API', 'Frontend Bug Fix'],
    'Development (Backend)': ['Database Schema Update', 'API Endpoint Creation', 'Performance Tuning'],
    'QA & Testing': ['Unit Testing', 'Integration Testing', 'UAT Coordination'],
    'Deployment & Release': ['Release Plan', 'Deployment Script', 'Post-Launch Monitoring Setup'],
    'Marketing & Promotion': ['Campaign Launch', 'Ad Spend Optimization', 'Performance Tracking'],
    'Analytics & Reporting': ['Data Collection Setup', 'Dashboard Build', 'Performance Report'],
    'PMO & Coordination': ['Stakeholder Sync', 'Risk Assessment', 'Dependency Management'],
    'Customer Support Prep': ['FAQ Document', 'Training Material', 'Support Handoff']
}


# --- Helper Functions for Data Generation ---

def generate_assignees_dim():
    """Generates the dim_assignees DataFrame."""
    assignees_data = []
    assignee_id_counter = 1
    for team in ASSIGNEE_TEAMS:
        for name in ASSIGNEE_NAMES_PER_TEAM[team]:
            assignee_id = f"ASS-{assignee_id_counter:03d}"
            assignee_id_counter += 1
            assignees_data.append({
                'assignee_id': assignee_id,
                'assignee_name': name,
                'assignee_team': team,
                'assignee_cost_per_hour': round(random.uniform(*ASSIGNEE_COST_PER_HOUR_RANGE), 2)
            })
    return pd.DataFrame(assignees_data)

def generate_task_types_dim():
    """Generates the dim_task_types DataFrame."""
    task_types_data = []
    for task_type_name in TASK_TYPES_LIST:
        task_types_data.append({
            'task_type_id': task_type_name, # Using name as ID for simplicity
            'task_type_name': task_type_name,
            'estimated_task_value': round(random.uniform(*ESTIMATED_TASK_VALUE_RANGE), 2)
        })
    return pd.DataFrame(task_types_data)

def generate_task_data(
    parent_id=None,
    level=1,
    existing_task_ids=None,
    assignees_df=None,
    task_types_df=None
):
    """Recursively generates a single task and its subtasks."""
    global task_id_counter
    task_id = f"T-{task_id_counter:05d}"
    task_id_counter += 1

    # Ensure existing_task_ids is a list
    if existing_task_ids is None:
        existing_task_ids = []

    # Select random assignee and task type from dimensions
    assignee_info = assignees_df.sample(1).iloc[0]
    assignee_id = assignee_info['assignee_id']

    task_type_info = task_types_df.sample(1).iloc[0]
    task_type = task_type_info['task_type_id']

    # Dates
    base_date = datetime.now() - timedelta(days=random.randint(90, 365)) # Tasks up to a year old
    date_created = base_date + timedelta(days=random.randint(0, 30))
    start_date = date_created + timedelta(days=random.randint(0, 7))
    due_date = start_date + timedelta(days=random.randint(5, 60)) # Due 5 to 60 days after start

    status = random.choice(STATUS_OPTIONS)
    percentage_completion = 0.0
    time_tracked_hours = 0.0
    date_completed = None

    if status == 'Completed':
        percentage_completion = 100.0
        # Completed date is usually on or before due date, but can be slightly after
        date_completed = due_date - timedelta(days=random.randint(-10, 5))
        time_tracked_hours = round(random.uniform(5.0, 40.0), 1)
    elif status == 'In Progress':
        percentage_completion = round(random.uniform(10.0, 90.0), 1)
        time_tracked_hours = round(random.uniform(1.0, 25.0), 1)
    elif status == 'Review':
        percentage_completion = 100.0 # Typically 100% for review
        time_tracked_hours = round(random.uniform(5.0, 35.0), 1)
        # Review tasks might have a completed date (initial work done) or not yet
        if random.random() < 0.7:
             date_completed = due_date - timedelta(days=random.randint(-10, 5))
    elif status == 'Blocked':
        percentage_completion = round(random.uniform(0.0, 80.0), 1)
        time_tracked_hours = round(random.uniform(0.0, 15.0), 1)
        # Blocked tasks might have a due date in the future, or already passed
        due_date = due_date + timedelta(days=random.randint(0, 30)) # Push due date out if blocked
    else: # To Do
        percentage_completion = 0.0
        time_tracked_hours = 0.0

    blocker_reason = random.choice(BLOCKER_REASONS) if status == 'Blocked' else None
    quality_review_status = None
    if status in ['Completed', 'Review'] and random.random() < 0.4: # 40% chance of review status
        quality_review_status = random.choice(QUALITY_REVIEW_STATUS_OPTIONS)

    last_updated_date = datetime.now() - timedelta(days=random.randint(0, 14)) # Last updated recently

    # Predecessor Task ID
    predecessor_task_id = None
    if level > 1 and len(existing_task_ids) > 5 and random.random() < CHANCE_TO_HAVE_PREDECESSOR:
        # Pick a random existing task that is not this task itself
        predecessor_task_id = random.choice([tid for tid in existing_task_ids if tid != task_id])

    # --- CORRECTED TASK NAME GENERATION ---
    task_name_suffix_options = TASK_NAME_SUFFIXES.get(task_type, ['Generic Task']) # Fallback
    task_name = f"{random.choice(task_name_suffix_options)} - {task_id}"
    # --- END CORRECTED TASK NAME GENERATION ---

    task_data = {
        'task_id': task_id,
        'parent_id': parent_id,
        'project_name': random.choice(PROJECT_NAMES),
        'task_name': task_name, # Use the newly generated realistic task name
        'status': status,
        'percentage_completion': percentage_completion,
        'date_created': date_created,
        'start_date': start_date,
        'due_date': due_date,
        'date_completed': date_completed,
        'time_tracked_hours': time_tracked_hours,
        'task_type': task_type,
        'complexity_points': random.randint(1, 5),
        'priority': random.choice(PRIORITY_OPTIONS),
        'blocker_reason': blocker_reason,
        'quality_review_status': quality_review_status,
        'last_updated_date': last_updated_date,
        'predecessor_task_id': predecessor_task_id,
        'assignee_id': assignee_id
    }

    all_tasks_data.append(task_data)
    existing_task_ids.append(task_id) # Add current task to list for future predecessors

    # Generate subtasks
    if level < MAX_SUB_LEVELS and random.random() < CHANCE_TO_HAVE_SUBTASKS:
        num_children = random.randint(*TASKS_PER_LEVEL_RANGE)
        for _ in range(num_children):
            generate_task_data(task_id, level + 1, existing_task_ids, assignees_df, task_types_df)


# --- Functions to Introduce Messiness ---

def introduce_missing_values(df, rates):
    """Introduces NaN values based on specified rates."""
    df_messy = df.copy()
    for col, rate in rates.items():
        if col in df_messy.columns:
            mask = np.random.rand(len(df_messy)) < rate
            df_messy.loc[mask, col] = np.nan
    return df_messy

def introduce_inconsistent_spellings(df, col_rates_map):
    """Introduces inconsistent spellings for categorical columns."""
    df_messy = df.copy()
    for col, rate in col_rates_map.items():
        if col in df_messy.columns:
            unique_values = df_messy[col].dropna().unique()
            if len(unique_values) > 0:
                for val in unique_values:
                    mask = (df_messy[col] == val) & (np.random.rand(len(df_messy)) < rate)
                    if col == 'status':
                        if val == 'To Do': df_messy.loc[mask, col] = random.choice(['todo', 'TO DO', 'To-Do'])
                        elif val == 'In Progress': df_messy.loc[mask, col] = random.choice(['in progress', 'Inprogress', 'IN PROGRESS'])
                        elif val == 'Completed': df_messy.loc[mask, col] = random.choice(['Done', 'completed.', 'COMPLETED'])
                        elif val == 'Blocked': df_messy.loc[mask, col] = random.choice(['blocked!', 'BLOCKED'])
                        elif val == 'Review': df_messy.loc[mask, col] = random.choice(['review', 'REVIEW'])
                    elif col == 'blocker_reason':
                        df_messy.loc[mask, col] = random.choice([val.lower(), val.upper(), val.replace(' ', ''), val + '!'])
                    elif col == 'priority':
                        if val == 'Critical': df_messy.loc[mask, col] = random.choice(['critical', 'CRITICAL'])
                        elif val == 'High': df_messy.loc[mask, col] = random.choice(['high', 'HIGH'])
                        elif val == 'Medium': df_messy.loc[mask, col] = random.choice(['medium', 'MED'])
                        elif val == 'Low': df_messy.loc[mask, col] = random.choice(['low', 'LOW'])
    return df_messy

def introduce_illogical_data(df):
    """Introduces illogical data points."""
    df_messy = df.copy()

    # Ensure date columns are datetime objects for consistency before operations
    # Use errors='coerce' to turn unparseable dates into NaT (Not a Time)
    df_messy['date_created'] = pd.to_datetime(df_messy['date_created'], errors='coerce')
    df_messy['date_completed'] = pd.to_datetime(df_messy['date_completed'], errors='coerce')
    df_messy['due_date'] = pd.to_datetime(df_messy['due_date'], errors='coerce')
    df_messy['last_updated_date'] = pd.to_datetime(df_messy['last_updated_date'], errors='coerce')


    # Completed but not 100%
    mask = (df_messy['status'] == 'Completed') & (np.random.rand(len(df_messy)) < ILLOGICAL_DATA_RATES['completed_but_not_100_percent'])
    df_messy.loc[mask, 'percentage_completion'] = round(random.uniform(50.0, 99.9), 1)

    # In Progress but 100%
    mask = (df_messy['status'] == 'In Progress') & (np.random.rand(len(df_messy)) < ILLOGICAL_DATA_RATES['in_progress_but_100_percent'])
    df_messy.loc[mask, 'percentage_completion'] = 100.0

    # Completed date before created date
    mask = (df_messy['status'] == 'Completed') & (np.random.rand(len(df_messy)) < ILLOGICAL_DATA_RATES['completed_before_created'])
    # Ensure date_created and date_completed are not NaT before comparison
    valid_dates_mask = df_messy['date_created'].notna() & df_messy['date_completed'].notna()
    df_messy.loc[mask & valid_dates_mask, 'date_completed'] = df_messy.loc[mask & valid_dates_mask, 'date_created'] - timedelta(days=random.randint(1, 30))

    # Overdue but no blocker reason (if status is Blocked and due date passed)
    # Ensure date columns are datetime objects for comparison (already done at start of function)
    overdue_blocked_mask = (df_messy['status'] == 'Blocked') & \
                           (df_messy['due_date'].notna()) & \
                           (df_messy['last_updated_date'].notna()) & \
                           (df_messy['due_date'] < df_messy['last_updated_date'])

    # Then, for a subset of these, remove the blocker reason
    mask_to_remove_blocker = overdue_blocked_mask & (np.random.rand(len(df_messy)) < ILLOGICAL_DATA_RATES['overdue_but_not_blocked'])
    df_messy.loc[mask_to_remove_blocker, 'blocker_reason'] = np.nan

    return df_messy


# --- Main Generation Process ---

# 1. Generate Dimension Tables
assignees_df = generate_assignees_dim()
task_types_df = generate_task_types_dim()

# 2. Generate Clean ClickUp Tasks Data
task_id_counter = 1
all_tasks_data = []
existing_task_ids = [] # To keep track of IDs for predecessor assignments

print("Generating clean dummy data...")
for _ in range(NUM_TOP_LEVEL_TASKS):
    generate_task_data(
        assignees_df=assignees_df,
        task_types_df=task_types_df,
        existing_task_ids=existing_task_ids # Pass the list by reference
    )

df_clean = pd.DataFrame(all_tasks_data)

# Convert dates to string format for CSV export (or keep as datetime for direct analysis)
# NOTE: This conversion happens *after* df_clean is created and before df_messy is derived.
# The introduce_illogical_data function now handles its own datetime conversion.
df_clean['date_created'] = df_clean['date_created'].dt.strftime('%Y-%m-%d')
df_clean['start_date'] = df_clean['start_date'].dt.strftime('%Y-%m-%d').fillna('')
df_clean['due_date'] = df_clean['due_date'].dt.strftime('%Y-%m-%d').fillna('')
df_clean['date_completed'] = df_clean['date_completed'].dt.strftime('%Y-%m-%d').fillna('')
df_clean['last_updated_date'] = df_clean['last_updated_date'].dt.strftime('%Y-%m-%d').fillna('')


# 3. Introduce Messiness for Version A
print("Introducing messiness for Version A...")
df_messy = df_clean.copy()

# Introduce missing values
df_messy = introduce_missing_values(df_messy, MISSING_VALUE_RATES)

# Introduce inconsistent spellings
df_messy = introduce_inconsistent_spellings(df_messy, INCONSISTENT_SPELLING_RATES)

# Introduce illogical data
df_messy = introduce_illogical_data(df_messy)

# Ensure 'parent_id' and 'predecessor_task_id' are string type for consistency,
# and replace NaN with empty string for CSV export if needed.
df_messy['parent_id'] = df_messy['parent_id'].fillna('').astype(str)
df_messy['predecessor_task_id'] = df_messy['predecessor_task_id'].fillna('').astype(str)
df_clean['parent_id'] = df_clean['parent_id'].fillna('').astype(str)
df_clean['predecessor_task_id'] = df_clean['predecessor_task_id'].fillna('').astype(str)


# --- Save to CSV Files ---
output_dir = './' # You can change this to a specific directory if needed

# Save dimension tables (clean)
assignees_df.to_csv(f'{output_dir}dim_assignees.csv', index=False)
task_types_df.to_csv(f'{output_dir}dim_task_types.csv', index=False)

# Save messy clickup tasks data (Version A)
df_messy.to_csv(f'{output_dir}dummy_clickup_tasks_messy.csv', index=False)

# NEW: Save clean clickup tasks data (Version B)
df_clean.to_csv(f'{output_dir}dummy_clickup_tasks_clean.csv', index=False)


print("\n--- Generation Complete ---")
print(f"Generated {len(df_messy)} messy tasks in 'dummy_clickup_tasks_messy.csv'")
print(f"Generated {len(df_clean)} clean tasks in 'dummy_clickup_tasks_clean.csv'") # Added clean count
print(f"Generated {len(assignees_df)} assignees in 'dim_assignees.csv'")
print(f"Generated {len(task_types_df)} task types in 'dim_task_types.csv'")

print("\nFirst 10 rows of MESSY data:")
print(df_messy.head(10))

print("\nMessy Data Info (check for NaNs and data types):")
print(df_messy.info())


Generating clean dummy data...
Introducing messiness for Version A...

--- Generation Complete ---
Generated 273 messy tasks in 'dummy_clickup_tasks_messy.csv'
Generated 273 clean tasks in 'dummy_clickup_tasks_clean.csv'
Generated 12 assignees in 'dim_assignees.csv'
Generated 12 task types in 'dim_task_types.csv'

First 10 rows of MESSY data:
   task_id parent_id                                     project_name  \
0  T-00001            Holiday Marketing Campaign - Social Media Focus   
1  T-00002   T-00001            Customer Onboarding Flow Optimization   
2  T-00003                 Q3 Product Feature Launch - AI Integration   
3  T-00004   T-00003            Customer Onboarding Flow Optimization   
4  T-00005   T-00004       Q3 Product Feature Launch - AI Integration   
5  T-00006   T-00003       Q3 Product Feature Launch - AI Integration   
6  T-00007   T-00006            Customer Onboarding Flow Optimization   
7  T-00008   T-00006     Internal Tool Development - Automation Suite  